In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from hmmlearn import hmm
from sklearn.preprocessing import KBinsDiscretizer, StandardScaler

# --------------------------------------
# 🎲 Simulate temperature and humidity based on hidden weather states
np.random.seed(42)

states = ['Sunny', 'Cloudy', 'Rainy']
n_states = len(states)

obs = []
true_states = []

for _ in range(1000):
    state = np.random.choice([0, 1, 2], p=[0.5, 0.3, 0.2])
    true_states.append(state)

    if state == 0:  # Sunny
        temp = np.random.normal(30, 3)
        hum = np.random.normal(40, 5)
    elif state == 1:  # Cloudy
        temp = np.random.normal(22, 2)
        hum = np.random.normal(60, 10)
    else:  # Rainy
        temp = np.random.normal(18, 2)
        hum = np.random.normal(80, 5)

    obs.append([temp, hum])

obs = np.array(obs)
true_states = np.array(true_states)

# --------------------------------------
# 🔢 1. Discrete HMM using MultinomialHMM
print("\n=== Discrete HMM ===")

# Discretize continuous values (temperature & humidity) into bins
discretizer = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform')
obs_discrete = discretizer.fit_transform(obs).astype(int)

# Combine temp & humidity bins into a single feature
obs_seq = (obs_discrete[:, 0] * 5 + obs_discrete[:, 1]).reshape(-1, 1)

# Train Multinomial HMM
model_discrete = hmm.MultinomialHMM(n_components=n_states, n_iter=100, random_state=42)
model_discrete.fit(obs_seq)

# Predict hidden states
pred_states_discrete = model_discrete.predict(obs_seq)

# Accuracy of discrete HMM
acc_discrete = np.mean(pred_states_discrete == true_states)
print(f"Discrete HMM Accuracy: {acc_discrete:.2f}")

# --------------------------------------
# 📈 2. Continuous HMM using GaussianHMM
print("\n=== Continuous HMM ===")

# Normalize the data for Gaussian HMM
scaler = StandardScaler()
obs_scaled = scaler.fit_transform(obs)

# Train Gaussian HMM
model_cont = hmm.GaussianHMM(n_components=n_states, n_iter=100, covariance_type='full', random_state=42)
model_cont.fit(obs_scaled)

# Predict hidden states
pred_states_cont = model_cont.predict(obs_scaled)

# Accuracy of continuous HMM
acc_cont = np.mean(pred_states_cont == true_states)
print(f"Continuous HMM Accuracy: {acc_cont:.2f}")
